# Hello GERD

## Getting started with GERD

In this notebook, we will start with basic features. If you run this notebook on binder, all required dependencies should already be installed. If you run this notebook locally, we'd recommend to create a virtual environment, install poetry and let peotry handle the installation of dependencies:

```shell
!pip install poetry
!poetry install --extras cpu --extras gguf
```

Before we start, we need to let Python know where to find `gerd`.
This is not necessary if you have used poetry as mentioned above since poetry will install `gerd` 'editable' and thus `import gerd` can be used without further configuration.

In [ ]:
import sys

sys.path.append("..")  # the project's root directory

First, let's load a `GenerationConfig` with `load_get_config`. `load_get_config` accepts a path to a config file or a config name as a string. The later will search for a config file with this name in the `config/` project directory. The code below will load `config/hello.yml`.

In [ ]:
from gerd.config import load_gen_config

config = load_gen_config("hello")

The `config` contains LLM model information which `gerd` (or more precisely, the `transformers` or `llama-cpp-python` library) will use to load a large language model. If this model has been downloaded before, it will be reused. Otherwise, it will be downloaded first. Note that since binder notebooks do not cache LLMs, you need to download them whenever you restart a notebook.

### Optional: Use GGUF model

The used LLM for this example is already quite small.
However, in a VM or container it still may take some time to process in- and output.
You could opt for a quantized model by uncomment the code below BEFORE you continue.
If you have already executed code below, just restart the Jupyter kernel and start over.

A quantized model provided reduces complexity and file size, increases processing speed but reduces accuracy.
For larger models, *some* quantization might not reduce output quality significantly but notably reduce file size and processing speed.
However, for smaller models with reduced parameter count, event slight quantization might make a model unusable.
Also note that `gerd` will use [llama.cpp](https://github.com/ggerganov/llama.cpp) instead of [transformers](https://github.com/huggingface/transformers) when you want to use quantized models.

In [ ]:
# # Uncomment if you want to use a GGUF
# # (quantized and thus smaller but less accurate) LLM.

# config.model.name = "Qwen/Qwen2.5-0.5B-Instruct-GGUF"
# config.model.file = "qwen2.5-0.5b-instruct-q5_k_m.gguf"  # 1 GB -> 522 MB
# # config.model.file = "qwen2.5-0.5b-instruct-q2_k.gguf"  # 415 MB; smallest version

Next, we will pass the configuration to a `gerd` LLM service. We use the `ChatService` which can be used for chat-like interaction but also to generate texts. 

In [ ]:
from gerd.gen.chat_service import ChatService

chat = ChatService(config)

Now let's use the chat service and submit a message to the LLM.
We do not pass messages directly but provide a dictionary with parameters to `submit_user_message`.
How and which parameters will be processed depends on the current prompt settings.
We can have a look at the current prompt by inspecting `config`.

In [ ]:
config.model.prompt_config.text

The initial prompt contains one variable argument `{word}` which is replaced with 'teleportation' by the call below and then passed to the LLM.
Note that executing this line of code may take a while on binder notebooks since the LLM is pretty demanding, especially when processed on the CPU.

If you have no more patience and haven't opted for the `GGUF` version of the model above, you might want to restart the Jupyter kernel and try again with the quantized LLM, either the recommended version, ending with `q5_k_m` or the even smaller version with the `q2_k` suffix.
On Binder, even a GGUF-quantized model may take some time to return though.

In [ ]:
res = chat.submit_user_message({"word": "teleportation"})
res.text

If you think the answer feels somewhat weird, you are correct.
`config` also provides an initial prompt with e.g. a so called 'system prompt' that adds some context and rules to the LLM interaction.
A `gerd` system prompt consists of a list of roles and `PromptConfig` objects.
This example provides one tuple with the role 'system' and a prompt as seen below.
Of course, this can be adjusted, too.

In [ ]:
config.model.prompt_setup[0][1].text

We can change prompt configurations without the need to reload an LLM.
The code below will replace the **user** prompt with an empty string.
`message` will be passed without modifications to the LLM.
However, the system prompt or prompt setup is still active and will influence the answer accordingly.

In [ ]:
from gerd.models.model import PromptConfig

chat.set_prompt_config(PromptConfig.model_validate({"text": "{message}"}))
res = chat.submit_user_message({"message": "Hello! What is one plus one?"})
res.text

Furthermore, the previous conversation will be passed as well.
Chat history can be reset with `chat.reset()`.

In [ ]:
chat.reset()